## Playground

In [1]:
# https://biopython.org/wiki/GFF_Parsing

from typing import List, Dict, Iterable, Tuple
import pprint
import itertools as it
import collections

import Bio
from Bio.SeqFeature import SeqFeature, ExactPosition, BeforePosition, AfterPosition
from Bio.SeqFeature import SimpleLocation, CompoundLocation
from BCBio.GFF import GFFExaminer
from BCBio import GFF

# in_file = "../samples/prokka.gff3"
# in_file = "../samples/augustus.gff3"
in_file = "../samples/maker.gff3"


In [2]:
recs = list(GFF.parse(in_file))
pprint.pp(recs)

[SeqRecord(seq=UnknownSeq(1274862, character='?'), id='4', name='<unknown name>', description='<unknown description>', dbxrefs=[])]


In [3]:
entry = recs[0]
pprint.pp(entry)

SeqRecord(seq=UnknownSeq(1274862, character='?'), id='4', name='<unknown name>', description='<unknown description>', dbxrefs=[])


In [4]:
f = entry.features[0]
f

SeqFeature(SimpleLocation(ExactPosition(53460), ExactPosition(64200), strand=-1), type='gene', id='maker-4-est_gff_Cufflinks-gene-0.0')

In [5]:
f.qualifiers

{'ID': ['maker-4-est_gff_Cufflinks-gene-0.0'],
 'Name': ['maker-4-est_gff_Cufflinks-gene-0.0'],
 'score': ['7.864828'],
 'source': ['maker']}

## Formatting `SeqRecord` as DDBJ annotation table

In [6]:
len(recs)

1

In [7]:
collections.Counter(f.type for f in recs[0].features) 

Counter({'gene': 69})

In [8]:
a_gene = recs[0].features[0]
collections.Counter(f.type for f in a_gene.sub_features)

Counter({'mRNA': 1})

In [9]:
a_transcript = a_gene.sub_features[0]
collections.Counter(f.type for f in a_transcript.sub_features)

Counter({'exon': 5, 'five_prime_UTR': 1, 'CDS': 5, 'three_prime_UTR': 1})

In [10]:
a_transcript.sub_features

[SeqFeature(SimpleLocation(ExactPosition(63539), ExactPosition(64200), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:4'),
 SeqFeature(SimpleLocation(ExactPosition(57141), ExactPosition(61911), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:3'),
 SeqFeature(SimpleLocation(ExactPosition(56499), ExactPosition(57083), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:2'),
 SeqFeature(SimpleLocation(ExactPosition(53816), ExactPosition(53999), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:1'),
 SeqFeature(SimpleLocation(ExactPosition(53460), ExactPosition(53751), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:0'),
 SeqFeature(SimpleLocation(ExactPosition(64050), ExactPosition(64200), strand=-1), type='five_prime_UTR', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:five_prime_utr'),
 SeqFeature(SimpleLocation(ExactPosition(63539), ExactPosition(64050),

In [11]:
target_type = "CDS"
target_features = [f for f in recs[0].features[0].sub_features[0].sub_features if f.type == target_type]
locs = [f.location for f in target_features]
compound_loc = CompoundLocation(locs)
feature_packed = SeqFeature(compound_loc, target_type)
feature_packed

SeqFeature(CompoundLocation([SimpleLocation(ExactPosition(63539), ExactPosition(64050), strand=-1), SimpleLocation(ExactPosition(57141), ExactPosition(61911), strand=-1), SimpleLocation(ExactPosition(56499), ExactPosition(57083), strand=-1), SimpleLocation(ExactPosition(53816), ExactPosition(53999), strand=-1), SimpleLocation(ExactPosition(53643), ExactPosition(53751), strand=-1)], 'join'), type='CDS', location_operator='join')

In [12]:
print(compound_loc)

join{[63539:64050](-), [57141:61911](-), [56499:57083](-), [53816:53999](-), [53643:53751](-)}


In [13]:
start_pos = AfterPosition(5)
end_pos = BeforePosition(10)
my_location = SimpleLocation(start_pos, end_pos)

In [14]:
str(my_location.start)

'>5'

In [15]:
target_features[2].qualifiers

{'ID': ['maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:cds'],
 'Parent': ['maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1'],
 'source': ['maker'],
 'phase': ['2']}

In [16]:
x = compound_loc.parts[0]

In [17]:
type(x.start)

Bio.SeqFeature.ExactPosition

In [18]:
compound_loc.parts

[SimpleLocation(ExactPosition(63539), ExactPosition(64050), strand=-1),
 SimpleLocation(ExactPosition(57141), ExactPosition(61911), strand=-1),
 SimpleLocation(ExactPosition(56499), ExactPosition(57083), strand=-1),
 SimpleLocation(ExactPosition(53816), ExactPosition(53999), strand=-1),
 SimpleLocation(ExactPosition(53643), ExactPosition(53751), strand=-1)]

In [19]:
entry

SeqRecord(seq=UnknownSeq(1274862, character='?'), id='4', name='<unknown name>', description='<unknown description>', dbxrefs=[])

## Testing translators in `src`

In [20]:
import json
import pathlib
import sys

pwd = pathlib.Path.cwd().parent
sys.path.append(str(pwd))

import src.translators as translators

In [21]:
collections.Counter(f.type for f in entry.features[0].sub_features[0].sub_features)

Counter({'exon': 5, 'five_prime_UTR': 1, 'CDS': 5, 'three_prime_UTR': 1})

In [22]:
paths = ["../src/translate_qualifiers.yaml"]
qualifier_converter = translators.TranslateQualifiers(paths)

paths = ["../src/translate_features.yaml"]
paths = [pathlib.Path(p).absolute() for p in paths]
feature_converter = translators.TranslateFeatures(paths)

entry = qualifier_converter.run(entry)
entry = feature_converter.run(entry)

In [23]:
cds = [f for f in entry.features[0].sub_features[0].sub_features if f.type == "CDS"]
cds[0].qualifiers

{'note': ['ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:cds', 'source:maker'],
 'Parent': ['maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1'],
 'codon_start': ['0']}

In [24]:
collections.Counter(f.type for f in entry.features[0].sub_features[0].sub_features)

Counter({'exon': 5, "5'UTR": 1, 'CDS': 5, "3'UTR": 1})

## DDBJ formatting

In [25]:
import pathlib
import sys

pwd = pathlib.Path.cwd().parent
sys.path.append(str(pwd))

import src.formatter
tbl = src.formatter.record_to_ddbj_table(entry)
s = src.formatter.table_to_tsv(tbl)

with open("ddbj_table.txt", "w") as f:
    print(s, file=f)

In [26]:
!head -34 "./ddbj_table.txt"

4	gene	complement(53460..64200)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0
			note	source:maker
	mRNA	complement(53460..64200)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1
			note	source:maker
	exon	complement(63539..64200)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:4
			note	source:maker
			Parent	maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1
	exon	complement(57141..61911)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:3
			note	source:maker
			Parent	maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1
	exon	complement(56499..57083)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:2
			note	source:maker
			Parent	maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1
	exon	complement(53816..53999)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:1
			note	source:maker
			Parent	maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1
	exon	complement(53460..53751)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:0
			note	source:maker
			Parent	maker-4-est_gff_Cufflinks-gene-0.0-mRNA

In [27]:
entry.features[0].sub_features[0].sub_features

[SeqFeature(SimpleLocation(ExactPosition(63539), ExactPosition(64200), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:4'),
 SeqFeature(SimpleLocation(ExactPosition(57141), ExactPosition(61911), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:3'),
 SeqFeature(SimpleLocation(ExactPosition(56499), ExactPosition(57083), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:2'),
 SeqFeature(SimpleLocation(ExactPosition(53816), ExactPosition(53999), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:1'),
 SeqFeature(SimpleLocation(ExactPosition(53460), ExactPosition(53751), strand=-1), type='exon', id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:0'),
 SeqFeature(SimpleLocation(ExactPosition(64050), ExactPosition(64200), strand=-1), type="5'UTR", id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:five_prime_utr'),
 SeqFeature(SimpleLocation(ExactPosition(63539), ExactPosition(64050), strand=-

## `assembly_gap`

In [28]:
from Bio.Seq import Seq
import Bio.SeqIO
from typing import List, Dict, Iterable, Tuple
import gzip

# https://www.ddbj.nig.ac.jp/news/ja/2021-08-10_3.html
fasta_path = "./fasta_na.BOUE01000001.txt.gz"

In [29]:
with gzip.open(fasta_path, 'rt') as f:
    seqs = Bio.SeqIO.parse(f, "fasta")
    for s in seqs:
        print(s.id)

BOUE01000001|BOUE01000001.1


In [31]:
src.translators.get_assembly_gap(s.seq)

[(848852, 848861),
 (1037995, 1038004),
 (1268390, 1268399),
 (1320750, 1320759),
 (1326129, 1326138),
 (1368923, 1368932),
 (1369648, 1369657),
 (1407014, 1407023),
 (1547450, 1547459),
 (1608547, 1608556),
 (1845750, 1845759),
 (2092435, 2092444),
 (2093497, 2093506),
 (2222806, 2222815),
 (2226754, 2226763),
 (2229737, 2229746),
 (2337459, 2337468),
 (2361686, 2361695),
 (2372429, 2372438),
 (2373972, 2373981),
 (2376826, 2378764),
 (2434780, 2434789),
 (2446911, 2446920),
 (2725397, 2725406),
 (2732641, 2732650),
 (2823806, 2823815),
 (2901099, 2901108),
 (2902164, 2902173),
 (2905544, 2905553),
 (2909241, 2909250),
 (2915487, 2915496),
 (2917500, 2917509),
 (3133148, 3133157),
 (3545811, 3545820),
 (3618053, 3618062),
 (3626493, 3626502),
 (3632244, 3632253),
 (3636722, 3636731),
 (3721270, 3721279),
 (3726246, 3726255),
 (4021210, 4021219),
 (4022548, 4022557),
 (4024195, 4024204),
 (4052792, 4052801),
 (4110186, 4110195),
 (4110492, 4110501),
 (4114367, 4114376),
 (4115237, 4115

In [32]:
s = Seq("atatnnngattacanccc")
src.translators.get_assembly_gap(s)

[(5, 7), (15, 15)]

## Load `COMMON` from YAML

In [33]:
from typing import Any, Dict, List, Tuple, Iterable, Union
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature
import yaml

path = "../samples/common.yaml"
common = src.formatter.load_common(path)
common

SeqRecord(seq='', id='COMMON', name='<unknown name>', description='<unknown description>', dbxrefs=[])

In [34]:
s = src.formatter.table_to_tsv(src.formatter.record_to_ddbj_table(common))
print(s)

COMMON	SUBMITTER		ab_name	Robertson,G.R.
			ab_name	Mishima,H.
			contact	Hanako Mishima
			email	mishima@ddbj.nig.ac.jp
			phone	81-55-981-6853
			institute	National Institute of Genetics
			country	Japan
			city	Mishima
			street	Yata 1111
			zip	411-8540
	REFERENCE		title	Mouse Genome Sequencing
			ab_name	Robertson,G.R.
			ab_name	Mishima,H
			year	2017
			status	Unpublished
	COMMENT		line	Please visit our website URL
			line	http://www.ddbj.nig.ac.jp/


In [35]:
fs = common.features
fs

[SeqFeature(None, type='SUBMITTER'),
 SeqFeature(None, type='REFERENCE'),
 SeqFeature(None, type='COMMENT')]

In [36]:
fs[2].qualifiers

{'line': ['Please visit our website URL', 'http://www.ddbj.nig.ac.jp/']}

# Joining features

Try joining mRNA and CDS (and more?)

In [37]:
import src.translators
import src.formatter
    
entry_updated = src.translators.join_features(entry)
tbl = src.formatter.record_to_ddbj_table(entry_updated)
s = src.formatter.table_to_tsv(tbl)

with open("ddbj_table_updated.txt", "w") as f:
    print(s, file=f)

In [38]:
!head -40 "./ddbj_table_updated.txt"

4	gene	complement(53460..64200)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0
			note	source:maker
	mRNA	complement(53460..64200)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1
			note	source:maker
	exon	complement(63539..64200)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:4
			note	source:maker
			Parent	maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1
	exon	complement(57141..61911)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:3
			note	source:maker
			Parent	maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1
	exon	complement(56499..57083)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:2
			note	source:maker
			Parent	maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1
	exon	complement(53816..53999)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:1
			note	source:maker
			Parent	maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1
	exon	complement(53460..53751)	note	ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:exon:0
			note	source:maker
			Parent	maker-4-est_gff_Cufflinks-gene-0.0-mRNA

In [39]:
entry_updated.features[0].sub_features[0].sub_features[6]

SeqFeature(CompoundLocation([SimpleLocation(ExactPosition(63539), ExactPosition(64050), strand=-1), SimpleLocation(ExactPosition(57141), ExactPosition(61911), strand=-1), SimpleLocation(ExactPosition(56499), ExactPosition(57083), strand=-1), SimpleLocation(ExactPosition(53816), ExactPosition(53999), strand=-1), SimpleLocation(ExactPosition(53643), ExactPosition(53751), strand=-1)], 'join'), type='CDS', location_operator='join')

In [40]:
entry.features[0].sub_features[0].sub_features[7]

SeqFeature(SimpleLocation(ExactPosition(53460), ExactPosition(53643), strand=-1), type="3'UTR", id='maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:three_prime_utr')

In [41]:
entry.features[0].sub_features[0].sub_features[7].qualifiers

{'note': ['ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:three_prime_utr',
  'source:maker'],
 'Parent': ['maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1']}

In [42]:
entry_updated.features[0].sub_features[0].sub_features[7].qualifiers

{'note': ['ID:maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1:three_prime_utr',
  'source:maker'],
 'Parent': ['maker-4-est_gff_Cufflinks-gene-0.0-mRNA-1']}